In [2]:
import dotenv
dotenv.load_dotenv()

True

# Introduction to Chains

In this notebook we will go through the most popular usecase for LangChain which is building Chains.

### What is a Chain?

Chain consists of chardcoded list of steps, which one can later execute on custom input. 
Recap what we did in the previous notebook, first we formatted the prompt and then we send it to the model for processing.
This is the simples example of a chain. Later we will explore more complex ones.


### LangChain Expression Language (LCEL)

LCEL is a declarative way to easily write and compose custom chains.
Apart from enabling developers to do it in a very pythonic way, it has several other benefits.
Any Chain constructed this way will automatically have full sync, async, batch, and streaming support. Also any components that can be run in parallel automatically are.

Let's start with rewriting the chain from previous notebook:

In [3]:
### imports
from langchain.chat_models import ChatOpenAI 
from langchain.prompts import (
    ChatPromptTemplate, 
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
)
from langchain.prompts.chat import HumanMessage
from langchain.schema.output_parser import StrOutputParser

In [4]:
behaviour = "an expert at telling jokes"
topic = "bear"

chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("You are {behaviour}."),
        HumanMessage(content="Hi. What's your name?"),
        AIMessagePromptTemplate.from_template("Hello. I am {behaviour}. What can I do for you today?"),
        HumanMessagePromptTemplate.from_template("Tell me a joke about {topic}")
    ]
)

chat_api = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.0)

chat_chain = chat_prompt | chat_api | StrOutputParser()

res = chat_chain.invoke({"behaviour": behaviour, "topic": topic})
print(res)

Sure, here's a bear-themed joke for you:

Why don't bears wear shoes?

Because they have bear feet!


In [5]:
chat_chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'behaviour': {'title': 'Behaviour'},
  'topic': {'title': 'Topic'}}}

In [6]:
chat_chain.output_schema.schema()

{'title': 'StrOutputParserOutput', 'type': 'string'}

### Building blocks

As you can see, the code is cleaner and easy to understand.

Now let's dig deeper and understand how those building blocks can be chained together.

In [7]:
from langchain.schema.runnable import (
    RunnableLambda, 
    RunnableMap, 
    RunnableBranch,
    RunnablePassthrough
)
from langchain.schema.output_parser import StrOutputParser
from operator import itemgetter

`RunnableLambda` is a wrapper arounf arbitrary function which makes it compatible with Expression Language.

It is important to remember, that the function used in chain must accept only one argument. 
So if you have a function which takes more than one argument, you can refactor it, to take a dictionary of inputs as argument or call it from another function which does so.

In [8]:
chain = RunnableLambda(lambda x: x + 1) | (lambda x: x * 2) # this syntax also works
chain.invoke(1)

4

In [9]:
def my_custom_fn(a, b, c, d):
    return f"{a} {b} {c} {d}"

def fn_wrapper(inputs):
    a = inputs["a"]
    b = inputs["b"]
    c = inputs["c"]
    d = inputs["d"]
    return my_custom_fn(a, b, c, d)

# or using the inspect module
from inspect import signature
def fn_wrapper2(inputs):
    fn_sig = signature(my_custom_fn)
    return my_custom_fn(**{k: inputs[k] for k in fn_sig.parameters.keys()})

chain = RunnableLambda(fn_wrapper)
chain.invoke({"a": 1, "b": 2, "c": 3, "d": 4})

'1 2 3 4'

`RunnableMap` can execute multiple Runnables in parallel and return the output of these Runnables as a map.

Each element of the map will get a full copy of the input and all the outputs will get combined into single dictionary.

In [10]:
def f1(inputs):
    print(f"f1 inputs: {inputs}")
    return "f1"

def f2(inputs):
    print(f"f2 inputs: {inputs}")
    return ["f2", "f2"]

def f3(inputs):
    print(f"f3 inputs: {inputs}")
    return {"some_name": "f3"}

chain = RunnableMap({
    "f1 return value": f1,
    "f2 return value": f2,
    "f3 return value": f3
})
chain.invoke([1, 2, 3])

f1 inputs: [1, 2, 3]
f2 inputs: [1, 2, 3]
f3 inputs: [1, 2, 3]


{'f1 return value': 'f1',
 'f2 return value': ['f2', 'f2'],
 'f3 return value': {'some_name': 'f3'}}

In [11]:
chain2 = chain | {
    "copy nr. 1": lambda x: x,
    "copy nr. 2": lambda x: x,
}

chain2.invoke([1, 2, 3])

f1 inputs: [1, 2, 3]
f2 inputs: [1, 2, 3]
f3 inputs: [1, 2, 3]


{'copy nr. 1': {'f1 return value': 'f1',
  'f2 return value': ['f2', 'f2'],
  'f3 return value': {'some_name': 'f3'}},
 'copy nr. 2': {'f1 return value': 'f1',
  'f2 return value': ['f2', 'f2'],
  'f3 return value': {'some_name': 'f3'}}}

You can also use `itemgetter` to extract arguments from input dictionary.

In [12]:
def f1(x):
    return x + 1

def f2(y):
    return y + " world"

chain = RunnableMap({
    "f1": itemgetter("x") | RunnableLambda(f1),
    "f2": itemgetter("y") | RunnableLambda(f2),
})

chain.invoke({"x": 1, "y": "hello"})


{'f1': 2, 'f2': 'hello world'}

`RunnableMap` runs in parallel

In [13]:
import time
from operator import itemgetter

sleep_chain = RunnableMap({
    't1': itemgetter('t1') | RunnableLambda(time.sleep), 
    't2': itemgetter('t2') | RunnableLambda(time.sleep),
    't3': itemgetter('t3') | RunnableLambda(time.sleep),
})

start = time.time()
sleep_chain.invoke({'t1': 3, 't2': 3, 't3': 3})
end = time.time()   
print(end - start)

3.006687879562378


To be able to write runnable maps as Python dictionaries and chain them using pipe `|` operator, there must be at leas one instance of Runnable class in the chain.
For example, this chain will not work, insted the pipe operator will be interpreted as dict union operator:

In [21]:
my_failed_chain = {
    'some_fn': RunnableLambda(lambda x: x + 1),
} | {
    'some_other_fn': RunnableLambda(lambda x: x + 1),
} | {
    'yet_another_fn': RunnableLambda(lambda x: x + 1),
}

print(type(my_failed_chain))
print(my_failed_chain)

<class 'dict'>
{'some_fn': RunnableLambda(...), 'some_other_fn': RunnableLambda(...), 'yet_another_fn': RunnableLambda(...)}


In [23]:
my_working_chain = {
    'some_fn': RunnableLambda(lambda x: x + 1),
} | {
    'some_other_fn': RunnableLambda(lambda x: x + 1),
} | RunnableMap({
    'yet_another_fn': RunnableLambda(lambda x: x + 1),
})

print(type(my_working_chain))
print(my_working_chain)

<class 'langchain.schema.runnable.base.RunnableSequence'>
first={
  some_fn: RunnableLambda(...),
  some_other_fn: RunnableLambda(...)
} last={
  yet_another_fn: RunnableLambda(...)
}


`RunnablePassthrough` passes the whole input further.

It can be used to wrap a input into dictionary to provide keys necessary for some function, or make the input available in later chain steps.

In [15]:
def search(query):
    return "search results"

def combine_results(results):
    query = results["query"]
    search_results = results["search_results"]

    return f"results for '{query}': {search_results}"

chain = RunnableMap({
    "query": RunnablePassthrough(),
    "search_results": RunnableLambda(search),
}) | RunnableLambda(combine_results)

chain.invoke("I am looking for ...")

"results for 'I am looking for ...': search results"

`RunnableBranch` is used for conditional branching.

In [16]:
def handle_int(x):
    print("handling int")
    return x + 1

def handle_str(x):
    print("handling str")
    return x + " world"

def default_handler(x):
    print("handling default")
    return x

chain = RunnableBranch(
    (lambda x: isinstance(x, int), RunnableLambda(handle_int)),
    (lambda x: isinstance(x, str), RunnableLambda(handle_str)),
    default_handler
)

# or passing a function that handles routing
def route(inputs):
    if isinstance(inputs, int):
        return handle_int(inputs)
    elif isinstance(inputs, str):
        return handle_str(inputs)
    else:
        return default_handler(inputs)
    
chain2 = RunnableLambda(route)

chain.invoke(1), chain.invoke("hello"), chain2.invoke(1.0)

handling int
handling str
handling default


(2, 'hello world', 1.0)

### Exercise: 
##### Create a chain for chatting with the model and storing conversation history.

In [17]:
from typing import Dict
from langchain.memory import ConversationBufferMemory
from langchain.schema.messages import SystemMessage
from langchain.prompts import HumanMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder


prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful chatbot."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{question}"),
])
memory = ConversationBufferMemory(return_messages=True, memory_key="history")
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

def get_question_and_history(input_string: str) -> Dict[str, str]:

    return {
        "question": ...,
        "history": memory.load_memory_variables(inputs=None) # get list of history messages from memory
    }

def get_answer_and_pass_question_through(input_dict: Dict[str, str]) -> Dict[str, str]:

    get_answer_chain = ... # format prompt > call model > extract output from AI Message object

    return {
        "answer": get_answer_chain.invoke(...), # recall from earlier, that prompt accepts a dictionary with key names == template variables
        "question": ..., # pass question further 
    }

def update_history_and_return_answer(inputs: Dict[str, str]) -> str:
    # we use question/answer keys because they are required by the object, but we dont use it here
    memory.save_context({'question': ...}, {'answer': ...}) # store both question and answer in memory object, 
    
    return ... # return answer string

chat_chain = RunnableLambda(get_question_and_history) | RunnableLambda(get_answer_and_pass_question_through) | RunnableLambda(update_history_and_return_answer) | StrOutputParser()


print(chat_chain.invoke("Hi. What's your name?"))
print(chat_chain.invoke("What is 2+2?"))

# see list of history messages
print(...)

AttributeError: 'ellipsis' object has no attribute 'invoke'

### Solution

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful chatbot."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{question}"),
])
memory = ConversationBufferMemory(return_messages=True, memory_key="history")
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)


def get_question_and_history(input_str: str) -> Dict[str, str]:
    return {
        "question": input_str,
        "history": memory.load_memory_variables(inputs=None)['history'],
    }

def get_answer_and_pass_question_through(input_dict: Dict[str, str]) -> Dict[str, str]:
    get_answer_chain = prompt | model | StrOutputParser()
    return {
        "question": input_dict['question'],
        "answer": get_answer_chain.invoke(input_dict),
    }

def update_history_and_return_answer(inputs: Dict[str, str]) -> str:
    memory.save_context({'question': inputs['question']}, {'answer': inputs['answer']})

    return inputs['answer']

chat_chain = RunnableLambda(get_question_and_history) | RunnableLambda(get_answer_and_pass_question_through) | RunnableLambda(update_history_and_return_answer)

print(chat_chain.invoke("Hi. What's your name?"))
print(chat_chain.invoke("What is 2+2?"))

print(memory.load_memory_variables({})['history'])

Hello! I am a chatbot, so I don't have a personal name. You can simply refer to me as "Chatbot" or any other name you prefer. How can I assist you today?
The sum of 2 and 2 is 4.
[HumanMessage(content="Hi. What's your name?"), AIMessage(content='Hello! I am a chatbot, so I don\'t have a personal name. You can simply refer to me as "Chatbot" or any other name you prefer. How can I assist you today?'), HumanMessage(content='What is 2+2?'), AIMessage(content='The sum of 2 and 2 is 4.')]
